In [1]:
# task2.4. Реализовать на базе модели определения пола выявление с другим нормированием (относительно среднего значения). Также добавьте альтернативную функцию активации

In [2]:
import numpy as np

def relu(x, alpha = 0.2):
  return alpha * x * (x > 0)

def deriv_relu(x, alpha = 0.2):
  return (x > 0) * alpha

def mse_loss(y_true, y_pred):
  return ((y_true - y_pred) ** 2).mean()

class OurNeuralNetwork:
  def __init__(self):
    # Weights
    self.w1 = np.random.normal()
    self.w2 = np.random.normal()
    self.w3 = np.random.normal()
    self.w4 = np.random.normal()
    self.w5 = np.random.normal()
    self.w6 = np.random.normal()

    # Biases
    self.b1 = np.random.normal()
    self.b2 = np.random.normal()
    self.b3 = np.random.normal()

  def feedforward(self, x):
    # x is a numpy array with 2 elements.
    h1 = relu(self.w1 * x[0] + self.w2 * x[1] + self.b1)
    h2 = relu(self.w3 * x[0] + self.w4 * x[1] + self.b2)
    o1 = relu(self.w5 * h1 + self.w6 * h2 + self.b3)
    return o1

  def train(self, data, all_y_trues):
    '''
    - data is a (n x 2) numpy array, n = # of samples in the dataset.
    - all_y_trues is a numpy array with n elements.
      Elements in all_y_trues correspond to those in data.
    '''
    learn_rate = 0.01
    epochs = 50 # number of times to loop through the entire dataset

    for epoch in range(epochs):
      for x, y_true in zip(data, all_y_trues):
        # --- Do a feedforward (we'll need these values later)
        sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
        h1 = relu(sum_h1)

        sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
        h2 = relu(sum_h2)

        sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
        o1 = relu(sum_o1)
        y_pred = o1

        # --- Calculate partial derivatives.
        # --- Naming: d_L_d_w1 represents "partial L / partial w1"
        d_L_d_ypred = -2 * (y_true - y_pred)

        # Neuron o1
        d_ypred_d_w5 = h1 * deriv_relu(sum_o1)
        d_ypred_d_w6 = h2 * deriv_relu(sum_o1)
        d_ypred_d_b3 = deriv_relu(sum_o1)

        d_ypred_d_h1 = self.w5 * deriv_relu(sum_o1)
        d_ypred_d_h2 = self.w6 * deriv_relu(sum_o1)

        # Neuron h1
        d_h1_d_w1 = x[0] * deriv_relu(sum_h1)
        d_h1_d_w2 = x[1] * deriv_relu(sum_h1)
        d_h1_d_b1 = deriv_relu(sum_h1)

        # Neuron h2
        d_h2_d_w3 = x[0] * deriv_relu(sum_h2)
        d_h2_d_w4 = x[1] * deriv_relu(sum_h2)
        d_h2_d_b2 = deriv_relu(sum_h2)

        # --- Update weights and biases
        # Neuron h1
        self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
        self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
        self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

        # Neuron h2
        self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
        self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
        self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

        # Neuron o1
        self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
        self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
        self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

      # --- Calculate total loss at the end of each epoch
      if epoch % 10 == 0:
        y_preds = np.apply_along_axis(self.feedforward, 1, data)
        loss = mse_loss(all_y_trues, y_preds)
        print("Epoch %d loss: %.3f" % (epoch, loss))


In [3]:
import numpy as np

def normalize_data(data):
    mv = np.mean(data)
    return (data - mv) / mv


In [4]:
data = np.array([
  [0, 1],  # Alice
  [27, 8],   # Bob
  [19, 6],   # Charlie
  [-13, -4], # Diana
])
all_y_trues = np.array([
  1, # Alice
  0, # Bob
  0, # Charlie
  1, # Diana
])

network = OurNeuralNetwork()
network.train(data, all_y_trues)
emily = np.array([-7, -1])
frank = np.array([20, 4])
print("Emily: %.3f" % network.feedforward(emily))
print("Frank: %.3f" % network.feedforward(frank))

Epoch 0 loss: 0.502
Epoch 10 loss: 0.496
Epoch 20 loss: 0.491
Epoch 30 loss: 0.486
Epoch 40 loss: 0.483
Emily: -0.000
Frank: 0.096


In [5]:
# task2.5. Реализовать экстемумы через градиентный спуск: x**2+2x-6

In [6]:
def dJ(x):
    return 2 * x + 2

 # Функция потерь
def J(x):
    return (x ** 2 + 2 * x - 6)

x = 0.0
 # Записываем точку каждого падения градиента
history_x=[x]
i_iter=0.1
 # Скорость обучения
learn_rate = 0.2
 # Выполнить градиентную итерацию
while i_iter < 1e4:
    i_iter += 0.1
    dj = dJ(x)
    last_x = x
    x = x - learn_rate * dj
    history_x.append(x)

         # Если итерация достигнута точно, расчет завершен
    if abs(J(x)-J(last_x))<0.0000000001 :
        break

history_y = J(np.array(history_x))
print(f'Координата x в минимуме параболы равна {round(x, 1)}')

Координата x в минимуме параболы равна -1.0
